In [1]:
#import Libraries
import json
import requests
import pprint as pp
import pandas as pd
from sqlalchemy import create_engine,inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
#get data from API created by Manoel in C# web api and deployed at Azure
url = 'http://manoelburgos.azurewebsites.net/api/Crimes'


In [74]:
# get content in json format
json_api_value = requests.get(url).json()

In [76]:
#print result to know which key should be retrieved
pp.pprint(json_api_value)

[{'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D53',
  'Event_Unique_ID': '111878',
  'Homicide_Type': 'Other',
  'Hood_ID': '98',
  'ID': 1,
  'Lat': '43.6850281',
  'Long': '-79.3928528',
  'Neighbourhood': 'Rosedale-Moore Park (98)',
  'Occurrence_Date': '2004-01-03T05:00:00.000Z',
  'Occurrence_year': '2004'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D42',
  'Event_Unique_ID': '125755',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '137',
  'ID': 2,
  'Lat': '43.7815361',
  'Long': '-79.2349625',
  'Neighbourhood': 'Woburn (137)',
  'Occurrence_Date': '2004-01-08T05:00:00.000Z',
  'Occurrence_year': '2004'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D42',
  'Event_Unique_ID': '136086',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '132',
  'ID': 3,
  'Lat': '43.8108597',
  'Long': '-79.2068939',
  'Neighbourhood': 'Malvern (132)',
  'Occurrence_Date': '2004-01-08T05:00:00.000Z',
  'Occurrence_year': '2004'},
 {'City': 'Toronto',
  'Country

  'Occurrence_Date': '2004-10-15T04:00:00.000Z',
  'Occurrence_year': '2004'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D53',
  'Event_Unique_ID': '696852',
  'Homicide_Type': 'Other',
  'Hood_ID': '99',
  'ID': 48,
  'Lat': '43.7010231',
  'Long': '-79.3785858',
  'Neighbourhood': 'Mount Pleasant East (99)',
  'Occurrence_Date': '2004-10-27T04:00:00.000Z',
  'Occurrence_year': '2004'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D31',
  'Event_Unique_ID': '3795155',
  'Homicide_Type': 'Other',
  'Hood_ID': '25',
  'ID': 49,
  'Lat': '43.7393913',
  'Long': '-79.5132217',
  'Neighbourhood': 'Glenfield-Jane Heights (25)',
  'Occurrence_Date': '2010-05-29T04:00:00.000Z',
  'Occurrence_year': '2010'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D13',
  'Event_Unique_ID': '697185',
  'Homicide_Type': 'Other',
  'Hood_ID': '92',
  'ID': 50,
  'Lat': '43.6763077',
  'Long': '-79.4505997',
  'Neighbourhood': 'Corso Italia-Davenport (92)',
  'Occ

  'Neighbourhood': 'Church-Yonge Corridor (75)',
  'Occurrence_Date': '2005-05-14T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D41',
  'Event_Unique_ID': '863844',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '128',
  'ID': 88,
  'Lat': '43.7841072',
  'Long': '-79.2416306',
  'Neighbourhood': 'Agincourt South-Malvern West (128)',
  'Occurrence_Date': '2005-05-15T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D42',
  'Event_Unique_ID': '988453',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '136',
  'ID': 89,
  'Lat': '43.7581673',
  'Long': '-79.1930542',
  'Neighbourhood': 'West Hill (136)',
  'Occurrence_Date': '2005-05-18T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D31',
  'Event_Unique_ID': '781084',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '25',
  'ID': 90,
  'Lat': '43.7396736',
  'Long': '-79.5096817',

  'Neighbourhood': 'Cabbagetown-South St.James Town (71)',
  'Occurrence_Date': '2012-10-23T04:00:00.000Z',
  'Occurrence_year': '2012'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D43',
  'Event_Unique_ID': '4703830',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '135',
  'ID': 148,
  'Lat': '43.7858925',
  'Long': '-79.213562',
  'Neighbourhood': 'Morningside (135)',
  'Occurrence_Date': '2012-10-28T04:00:00.000Z',
  'Occurrence_year': '2012'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D32',
  'Event_Unique_ID': '4697103',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '31',
  'ID': 149,
  'Lat': '43.7208595',
  'Long': '-79.4493713',
  'Neighbourhood': 'Yorkdale-Glen Park (31)',
  'Occurrence_Date': '2012-10-29T04:00:00.000Z',
  'Occurrence_year': '2012'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D14',
  'Event_Unique_ID': '4613667',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '85',
  'ID': 150,
  'Lat': '43.6369286',
  'Long': '-79.4372

  'Neighbourhood': 'North St.James Town (74)',
  'Occurrence_Date': '2005-08-15T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D54',
  'Event_Unique_ID': '1072514',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '44',
  'ID': 208,
  'Lat': '43.7187347',
  'Long': '-79.3349533',
  'Neighbourhood': 'Flemingdon Park (44)',
  'Occurrence_Date': '2005-08-16T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D41',
  'Event_Unique_ID': '117846',
  'Homicide_Type': 'Other',
  'Hood_ID': '127',
  'ID': 209,
  'Lat': '43.7581253',
  'Long': '-79.2666245',
  'Neighbourhood': 'Bendale (127)',
  'Occurrence_Date': '2005-08-18T04:00:00.000Z',
  'Occurrence_year': '2005'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D22',
  'Event_Unique_ID': '1183577',
  'Homicide_Type': 'Other',
  'Hood_ID': '18',
  'ID': 210,
  'Lat': '43.6030045',
  'Long': '-79.5074844',
  'Neighbourhood': 

...

  'ID': 807,
  'Lat': '43.7395859',
  'Long': '-79.2228241',
  'Neighbourhood': 'Scarborough Village (139)',
  'Occurrence_Date': '2016-08-25T04:00:00.000Z',
  'Occurrence_year': '2016'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D12',
  'Event_Unique_ID': 'GO-20151352151',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '91',
  'ID': 808,
  'Lat': '43.6765747',
  'Long': '-79.4638901',
  'Neighbourhood': 'Weston-Pellam Park (91)',
  'Occurrence_Date': '2015-08-07T04:00:00.000Z',
  'Occurrence_year': '2015'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D12',
  'Event_Unique_ID': '4017400',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '113',
  'ID': 809,
  'Lat': '43.6973038',
  'Long': '-79.5094604',
  'Neighbourhood': 'Weston (113)',
  'Occurrence_Date': '2011-02-16T05:00:00.000Z',
  'Occurrence_year': '2011'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D43',
  'Event_Unique_ID': 'GO-20161506128',
  'Homicide_Type': 'Other',
  'Hood_ID': '139

  'ID': 867,
  'Lat': '43.8055801',
  'Long': '-79.2209244',
  'Neighbourhood': 'Malvern (132)',
  'Occurrence_Date': '2012-01-24T05:00:00.000Z',
  'Occurrence_year': '2012'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D14',
  'Event_Unique_ID': 'GO-20161613172',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '85',
  'ID': 868,
  'Lat': '43.6373482',
  'Long': '-79.4253387',
  'Neighbourhood': 'South Parkdale (85)',
  'Occurrence_Date': '2016-09-11T04:00:00.000Z',
  'Occurrence_year': '2016'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D31',
  'Event_Unique_ID': 'GO-20161625137',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '27',
  'ID': 869,
  'Lat': '43.7685204',
  'Long': '-79.5005569',
  'Neighbourhood': 'York University Heights (27)',
  'Occurrence_Date': '2016-09-13T04:00:00.000Z',
  'Occurrence_year': '2016'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D43',
  'Event_Unique_ID': 'GO-20161696689',
  'Homicide_Type': 'Stabbing',
  'Hood_

  'ID': 907,
  'Lat': '43.6606712',
  'Long': '-79.3788757',
  'Neighbourhood': 'Church-Yonge Corridor (75)',
  'Occurrence_Date': '2015-12-15T05:00:00.000Z',
  'Occurrence_year': '2015'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D31',
  'Event_Unique_ID': 'GO-20152149617',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '21',
  'ID': 908,
  'Lat': '43.7559776',
  'Long': '-79.5720749',
  'Neighbourhood': 'Humber Summit (21)',
  'Occurrence_Date': '2015-12-15T05:00:00.000Z',
  'Occurrence_year': '2015'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D52',
  'Event_Unique_ID': 'GO-20152123328',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '76',
  'ID': 909,
  'Lat': '43.6473351',
  'Long': '-79.3797302',
  'Neighbourhood': 'Bay Street Corridor (76)',
  'Occurrence_Date': '2015-12-11T05:00:00.000Z',
  'Occurrence_year': '2015'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D41',
  'Event_Unique_ID': 'GO-20152071129',
  'Homicide_Type': 'Other',
  '

  'Hood_ID': '112',
  'ID': 967,
  'Lat': '43.6883698',
  'Long': '-79.4828796',
  'Neighbourhood': 'Beechborough-Greenbrook (112)',
  'Occurrence_Date': '2017-06-07T04:00:00.000Z',
  'Occurrence_year': '2017'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D31',
  'Event_Unique_ID': 'GO-20171035511',
  'Homicide_Type': 'Shooting',
  'Hood_ID': '22',
  'ID': 968,
  'Lat': '43.7476883',
  'Long': '-79.5572815',
  'Neighbourhood': 'Humbermede (22)',
  'Occurrence_Date': '2017-06-11T04:00:00.000Z',
  'Occurrence_year': '2017'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D11',
  'Event_Unique_ID': 'GO-20171118832',
  'Homicide_Type': 'Stabbing',
  'Hood_ID': '93',
  'ID': 969,
  'Lat': '43.6672173',
  'Long': '-79.4441757',
  'Neighbourhood': 'Dovercourt-Wallace Emerson-Junction (93)',
  'Occurrence_Date': '2017-06-23T04:00:00.000Z',
  'Occurrence_year': '2017'},
 {'City': 'Toronto',
  'Country': 'Canada',
  'Division': 'D51',
  'Event_Unique_ID': 'GO-201711845

AttributeError: 'NoneType' object has no attribute 'head'

In [6]:
# open connection with the local DB (postgres)
rds_connection_string = "postgres:filho21@localhost:5432/Crimes"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [81]:
# identify the tables name
tables = engine.table_names()

In [82]:
# identify the internal columns of the table
inspector = inspect(engine)

In [86]:
#analyzing the 
for table in tables:
    columns = inspector.get_columns(table)
    for c in columns:
        print(f'table: {table} | column= {c["name"]} | type= {c["type"]}')

table: city | column= id | type= INTEGER
table: city | column= name | type= VARCHAR(50)
table: city | column= country | type= VARCHAR(50)
table: city | column= state | type= VARCHAR(50)
table: registercrime | column= id | type= INTEGER
table: registercrime | column= id_crime | type= INTEGER
table: registercrime | column= id_city | type= INTEGER
table: registercrime | column= day | type= INTEGER
table: registercrime | column= month | type= VARCHAR(10)
table: registercrime | column= year | type= INTEGER
table: registercrime | column= latitude | type= REAL
table: registercrime | column= longitude | type= REAL
table: crimestype | column= id | type= INTEGER
table: crimestype | column= type | type= VARCHAR(100)
table: census | column= id | type= INTEGER
table: census | column= id_city | type= INTEGER
table: census | column= year | type= INTEGER
table: census | column= population | type= REAL


In [87]:
# automap_base to map all the columns into object
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [88]:
#create a table's object
City = Base.classes.city
CrimeType = Base.classes.crimestype
RegisterCrime = Base.classes.registercrime

In [89]:
# query to avoid data duplicaton
query = session.query(City).filter(City.name=="Toronto").all()
print(query)
if query == []:
    city_one = City(name='Toronto', country="Canada", state="ON")
    session.add(city_one)
    session.commit()

In [91]:
# getting just a single value for each type 
dic_type ={}
for value in json_api_value:
    dic_type[value['Homicide_Type'].upper()]=value['Homicide_Type'].upper()
    

In [92]:
#printing to check results
print(dic_type)
dict_type2 = list(dic_type)
print(dict_type2)

{'OTHER': 'OTHER', 'SHOOTING': 'SHOOTING', 'STABBING': 'STABBING'}
['OTHER', 'SHOOTING', 'STABBING']


In [93]:
#check if the data is already there, then this avoid dupication
for data_value in dict_type2:
    query = session.query(CrimeType).filter(CrimeType.type ==data_value).all()
    if query == []:
        type_one = CrimeType(type=data_value)
        session.add(type_one)
        session.commit()

In [94]:
#deleting content of the table, after that insert 
engine.execute("delete FROM registercrime")
session.commit()

In [95]:
#inserting all data from json. 
for value in json_api_value:
#     find the id from the primery key to insert as foreign key
    id_city = session.query(City.id).filter(City.name==value['City']).first()
    id_crime = session.query(CrimeType.id).filter(CrimeType.type==value['Homicide_Type'].upper()).first() 
    complete_date = str(value['Occurrence_Date'])
    month =  complete_date[5:7]
    year =complete_date[0:4]
    day = complete_date[8:10]
    
    session.add(RegisterCrime(id_crime = id_crime, id_city = id_city, day=day, month=month, year=year,latitude = value['Lat'], longitude = value['Long']))
    session.commit()